In [ ]:
import geopandas as gpd
import rasterio
import rasterio.mask
import numpy as np
import os
# Function to extract statistics (mean, etc.) from raster within the boundary

# The script calculates the mean value of raster data (e.g., soil or environmental features) within each geometry (paddock boundaries)
# in a GeoDataFrame. The results are added to the GeoDataFrame as new columns, where each column corresponds 
# to a raster file's name (feature). The updated GeoDataFrame is then saved to a new GeoPackage file.

def extract_raster_stats(gdf, raster_path, column_prefix):
    # Open the raster file using rasterio
    with rasterio.open(raster_path) as src:
        for index, row in gdf.iterrows():
            # Get the geometry in the same CRS as the raster
            geom = [row['geometry']]
            try:
                out_image, out_transform = rasterio.mask.mask(src, geom, crop=True)
                out_image = out_image[0]  # Extract the first band if multiple
                # Filter out nodata values
                valid_pixels = out_image[out_image != src.nodata]

                if valid_pixels.size > 0:  # Ensure there are valid pixels to compute statistics
                    mean_value = np.nanmean(valid_pixels)
                    gdf.at[index, f"{column_prefix}_mean"] = mean_value
                else:
                    gdf.at[index, f"{column_prefix}_mean"] = np.nan
            except Exception as e:
                print(f"Error processing geometry {index}: {e}")
                gdf.at[index, f"{column_prefix}_mean"] = np.nan
                
    return gdf

# Load the GeoDataFrame containing the paddock boundaries
gdf = gpd.read_file(
    "C://Users//M.Ibrah//OneDrive - Department of Primary Industries And Regional Development//Desktop//Updated-Code-FoodAgility-Dec-24//Complete dataset//New_boundaries__Soil_data-12-2024-1.gpkg"
)

# Directory containing raster files
raster_folder = "F://Food Agility Data//Soil Features//DEM_Data//"
# List all .tif files in the raster folder
raster_files = [f for f in os.listdir(raster_folder) if f.endswith(".tif")]
# Loop over each raster file and process the data for each soil parameter
for raster_file in raster_files:
    # Derive the column prefix from the raster filename (without extension)
    column_name = os.path.splitext(raster_file)[0]
    raster_path = os.path.join(raster_folder, raster_file)
    print(f"Processing {column_name}...")
    gdf = extract_raster_stats(gdf, raster_path, column_name)

# Save the updated GeoDataFrame to a new file
output_path = "C://Users//M.Ibrah//OneDrive - Department of Primary Industries And Regional Development//Desktop//Updated-Code-FoodAgility-Dec-24//Complete dataset//New_boundaries__Soil_data-12-2024-1.gpkg"
gdf.to_file(output_path, driver="GPKG")
print(f"Updated GeoDataFrame saved to {output_path}.")


In [ ]:
import os
import numpy as np
import rasterio
import rasterio.mask

def create_feature_vector(gdf, raster_folder, feature_name):
    """
    Computes the mean value of all valid pixels within each paddock and assigns it to the GeoDataFrame.

    Parameters:
        gdf (GeoDataFrame): The GeoDataFrame containing paddock geometries and year attributes.
        raster_folder (str): Path to the folder containing raster files.
        feature_name (str): The feature name to associate the mean values.

    Returns:
        GeoDataFrame: Updated GeoDataFrame with the computed mean values for the feature.
    """
    # Sort GeoDataFrame by year
    gdf_sorted = gdf.sort_values(by="year").reset_index(drop=True)

    for year in range(2020, 2023):  # Adjust year range as needed
        raster_filename = f'{year}.{feature_name}.tif'
        raster_path = os.path.join(raster_folder, raster_filename)
        
        # Filter paddocks for the current year
        paddocks = gdf_sorted[gdf_sorted['year'] == year]

        with rasterio.open(raster_path) as src:
            for index, row in paddocks.iterrows():
                geom = [row.geometry]  # Extract geometry as a list
                try:
                    # Mask raster data to the paddock geometry
                    out_image, out_transform = rasterio.mask.mask(src, geom, crop=True)
                    out_image = out_image[0]  # Extract the first band if multiple

                    # Filter out nodata values
                    valid_pixels = out_image[out_image != src.nodata]

                    # Compute the mean value of valid pixels
                    if valid_pixels.size > 0:  # Ensure there are valid pixels to compute statistics
                        mean_value = np.nanmean(valid_pixels)
                        print(mean_value)
                    else:
                        mean_value = np.nan
                    
                    # Assign the mean value to the corresponding row in the GeoDataFrame
                    gdf_sorted.at[index, f"{feature_name}_mean"] = mean_value

                except Exception as e:
                    print(f"Error processing geometry {index}: {e}")
                    gdf_sorted.at[index, f"{feature_name}_mean"] = np.nan

    return gdf_sorted


In [ ]:
# Example usage of the function

import geopandas as gpd
import rasterio
import rasterio.mask
import numpy as np
import os

# Load the GeoDataFrame containing the paddock boundaries
gdf = gpd.read_file(
    "C://Users//M.Ibrah//OneDrive - Department of Primary Industries And Regional Development//Desktop//Updated-Code-FoodAgility-Dec-24//Complete dataset//New_boundaries__Soil_data-12-2024-1.gpkg"
)

gdf_updated = create_feature_vector(gdf, 'F://Food Agility Data//Soil Features//Soil_Albedo', 'Albedo')


# Save the updated GeoDataFrame to a new file
output_path = "C://Users//M.Ibrah//OneDrive - Department of Primary Industries And Regional Development//Desktop//Updated-Code-FoodAgility-Dec-24//Complete dataset//New_boundaries__Soil_data-12-2024-1.gpkg"
gdf_updated.to_file(output_path, driver="GPKG")
print(f"Updated GeoDataFrame saved to {output_path}.")



In [ ]:
import geopandas as gpd
import rasterio
import rasterio.mask
import numpy as np
import os


from scipy.ndimage import distance_transform_edt

def extract_raster_stats_with_interpolation(gdf, raster_path, column_prefix):
    with rasterio.open(raster_path) as src:
        for index, row in gdf.iterrows():
            geom = [row['geometry']]
            try:
                # Mask the raster with the geometry
                out_image, out_transform = rasterio.mask.mask(src, geom, crop=True)
                out_image = out_image[0]  # First band if multiple bands

                # Mask nodata values
                nodata = src.nodata
                valid_mask = out_image != nodata  # Mask of valid pixels

                if np.any(valid_mask):  # Check if there are valid pixels
                    # Replace NaN values with nearest neighbor interpolation
                    filled_image = replace_with_nearest(out_image, valid_mask)

                    # Compute statistics (mean)
                    mean_value = np.nanmean(filled_image)
                    gdf.at[index, f"{column_prefix}_mean"] = mean_value
                else:
                    print(f"No valid pixels for geometry {index}. Assigning NaN.")
                    gdf.at[index, f"{column_prefix}_mean"] = np.nan
            except rasterio.errors.RasterioIOError as e:
                print(f"RasterioIOError for geometry {index}. Skipping...")
                gdf.at[index, f"{column_prefix}_mean"] = np.nan
            except Exception as e:
                print(f"General error for geometry {index}: {e}")
                gdf.at[index, f"{column_prefix}_mean"] = np.nan
    return gdf

def replace_with_nearest(data, valid_mask):
    """
    Replace NaN or invalid values in the raster with the nearest valid pixel value.
    """
    # Create a distance map to the nearest valid pixel
    distances, nearest_indices = distance_transform_edt(~valid_mask, return_indices=True)

    # Replace invalid values with the nearest valid pixel value
    filled_data = data[tuple(nearest_indices)]
    filled_data[~valid_mask] = filled_data[tuple(nearest_indices)][~valid_mask]
    
    return filled_data



# Load the GeoDataFrame containing the paddock boundaries
gdf = gpd.read_file(
    "E://Updated-Rainfall-Zones-Analysis-Project//Complete dataset//New_boundaries_with_Climate_And_soil_data-12-24.gpkg"
)

# Directory containing raster files
raster_folder = "E://Food Agility Data//Soil Features//Clay"
# List all .tif files in the raster folder
raster_files = [f for f in os.listdir(raster_folder) if f.endswith(".tif")]
# Loop over each raster file and process the data for each soil parameter
for raster_file in raster_files:
    # Derive the column prefix from the raster filename (without extension)
    column_name = os.path.splitext(raster_file)[0]
    raster_path = os.path.join(raster_folder, raster_file)
    print(f"Processing {column_name}...")
    gdf = extract_raster_stats_with_interpolation(gdf, raster_path, column_name)

# Save the updated GeoDataFrame to a new file
output_path = "E://Updated-Rainfall-Zones-Analysis-Project//Complete dataset//New_boundaries_with_Climate_And_soil_data-12-2024.gpkg"
gdf.to_file(output_path, driver="GPKG")
print(f"Updated GeoDataFrame saved to {output_path}.")


In [ ]:
import geopandas as gpd
import rasterio
import rasterio.mask
import numpy as np

# Load the GeoDataFrame
gdf = gpd.read_file(
    "E://Updated-Rainfall-Zones-Analysis-Project//Complete dataset//New_boundaries_with_Climate_And_soil_data-12-24.gpkg"
)

# Test a single raster file
raster_path = "E://Food Agility Data//Soil Features//Clay//New folder//CLY.tif"

# Open the raster file and test on a single geometry
with rasterio.open(raster_path) as src:
    geom = [gdf.loc[0, 'geometry']]  # Select the first geometry
    out_image, out_transform = rasterio.mask.mask(src, geom, crop=True)
    out_image = out_image[0]
    valid_pixels = out_image[out_image != src.nodata]

    if valid_pixels.size > 0:
        mean_value = np.nanmean(valid_pixels)
        print(f"Mean Value: {mean_value}")
    else:
        print("No valid pixels in the selected geometry.")


In [ ]:
import geopandas as gpd

# Define the path to the GeoDataFrame file
file_path = "C://Users//M.Ibrah//OneDrive - Department of Primary Industries And Regional Development//Desktop//Updated-Code-FoodAgility-Dec-24//Complete dataset//New_boundaries__Soil_data-12-2024-1.gpkg"  # Replace with your file path

# Read the GeoDataFrame
try:
    gdf = gpd.read_file(file_path)
    
    # Print the column names
    print("Column names in the GeoDataFrame:")
    for column in gdf.columns:
        print(column)
except Exception as e:
    print(f"An error occurred: {e}")


In [ ]:
# Compute the Mode and Median of Paddocks for Color and Classification of soil

import geopandas as gpd
import rasterio
import rasterio.mask
import numpy as np
import os
from scipy import stats  # Import for mode calculation

# Function to extract statistics (median and mode) from raster within the boundary
def extract_raster_stats(gdf, raster_path, column_prefix):
    # Open the raster file using rasterio
    with rasterio.open(raster_path) as src:
        for index, row in gdf.iterrows():
            # Get the geometry in the same CRS as the raster
            geom = [row['geometry']]
            try:
                # Mask the raster to the geometry
                out_image, out_transform = rasterio.mask.mask(src, geom, crop=True)
                out_image = out_image[0]  # Extract the first band if multiple

                # Filter out nodata values
                valid_pixels = out_image[out_image != src.nodata]

                if valid_pixels.size > 0:  # Ensure there are valid pixels to compute statistics
                    # Compute median
                    median_value = np.nanmedian(valid_pixels)
                    gdf.at[index, f"{column_prefix}_median"] = median_value

                    # Compute mode (using scipy.stats.mode)
                    mode_result = stats.mode(valid_pixels, nan_policy='omit', keepdims=False)
                    mode_value = mode_result.mode if mode_result.count > 0 else np.nan
                    gdf.at[index, f"{column_prefix}_mode"] = mode_value
                else:
                    gdf.at[index, f"{column_prefix}_median"] = np.nan
                    gdf.at[index, f"{column_prefix}_mode"] = np.nan
            except Exception as e:
                print(f"Error processing geometry {index}: {e}")
                gdf.at[index, f"{column_prefix}_median"] = np.nan
                gdf.at[index, f"{column_prefix}_mode"] = np.nan
    return gdf

# Load the GeoDataFrame containing the paddock boundaries
gdf = gpd.read_file(
    "C://Users//M.Ibrah//OneDrive - Department of Primary Industries And Regional Development//Desktop//Updated-Code-FoodAgility-Dec-24//Complete dataset//New_boundaries__Soil_data-12-2024-1.gpkg"
)

# Directory containing raster files
raster_folder = "F://Food Agility Data//Soil Features//Soil Classification//"
# List all .tif files in the raster folder
raster_files = [f for f in os.listdir(raster_folder) if f.endswith(".tif")]

# Loop over each raster file and process the data for each soil parameter
for raster_file in raster_files:
    # Derive the column prefix from the raster filename (without extension)
    column_name = os.path.splitext(raster_file)[0]
    raster_path = os.path.join(raster_folder, raster_file)
    print(f"Processing {column_name}...")
    gdf = extract_raster_stats(gdf, raster_path, column_name)

# Save the updated GeoDataFrame to a new file
output_path = "C://Users//M.Ibrah//OneDrive - Department of Primary Industries And Regional Development//Desktop//Updated-Code-FoodAgility-Dec-24//Complete dataset//New_boundaries__Soil_data-12-2024-1.gpkg"
gdf.to_file(output_path, driver="GPKG")
print(f"Updated GeoDataFrame saved to {output_path}.")

In [ ]:
# Soil Moisture data extraction
def process_nc_file3(nc_file_path):
    # Open the .nc file using xarray
    with xr.open_dataset(nc_file_path) as dataset:
        # Extract monthly rainfall data
        # Extract the first variable name automatically
        var_name = list(dataset.data_vars.keys())[0]
        print(var_name)
        data = dataset[var_name]
        # Calculate the annual total rainfall (sum over time dimension)
        total_rain = data.mean(dim='time')
        
        # Extract spatial dimensions (lat, lon)
        lat = dataset['lat'].values
        lon = dataset['lon'].values
        
        # Calculate resolution from lat/lon data
        lat_res = abs(lat[1] - lat[0])
        lon_res = abs(lon[1] - lon[0])
        
        # Flip the data array along the latitude axis to match the raster orientation
        total_rain = np.flipud(total_rain.values)
        total_rain = interpolate_zeros_and_nans(total_rain)

        # print(f"Data shape: {data.shape}")
        
        # # Define transform (assuming data is a regular grid)
        # transform = from_origin(lon.min(), lat.max(), lon[1] - lon[0], lat[1] - lat[0])
        
        # Define transform for the raster file
        transform = from_origin(west=lon.min(), north=lat.max(), xsize=lon_res, ysize=lat_res)
        
        # Prepare the output raster file path
        output_file = os.path.splitext(nc_file_path)[0] + '.tif'
        
        # Save the data as a GeoTIFF file
        with rasterio.open(
            output_file, 
            'w', 
            driver='GTiff', 
            height=total_rain.shape[0], 
            width=total_rain.shape[1], 
            count=1, 
            dtype=np.float32, 
            crs='EPSG:4326',  # Assuming data is in WGS84
            transform=transform
        ) as dst:
            dst.write(total_rain.astype(np.float32), 1)
    
    print(f'Saved {output_file}')

In [ ]:
# Soil Moisture Data Extraction
def process_nc_file(nc_file_path):
    import xarray as xr
    import rasterio
    from rasterio.transform import from_origin
    import numpy as np
    import os

    def interpolate_zeros_and_nans(data_array):
        # Ensure the array is numeric before interpolation
        if not np.issubdtype(data_array.dtype, np.number):
            raise TypeError(f"Cannot interpolate non-numeric data type: {data_array.dtype}")
        # Replace NaNs and zeros with nearby values
        mask = np.isnan(data_array) | (data_array == 0)
        if not np.any(~mask):  # If all values are NaN or zero, skip interpolation
            return data_array
        filled_data = np.copy(data_array)
        filled_data[mask] = np.interp(
            np.flatnonzero(mask),
            np.flatnonzero(~mask),
            data_array[~mask]
        )
        return filled_data

    # Open the .nc file using xarray
    with xr.open_dataset(nc_file_path) as dataset:
        # Replace _FillValue with NaN
        data = dataset["sm_pct"].where(dataset["sm_pct"] != -999)

        # Calculate the annual mean soil moisture (mean over time dimension)
        total_moisture = data.mean(dim="time", skipna=True)

        # Extract spatial dimensions
        lat = dataset["latitude"].values
        lon = dataset["longitude"].values

        # Calculate resolution from lat/lon data
        lat_res = abs(lat[1] - lat[0])
        lon_res = abs(lon[1] - lon[0])

        # Flip the data array along the latitude axis to match the raster orientation
        total_moisture = np.flipud(total_moisture.values)
        total_moisture = interpolate_zeros_and_nans(total_moisture)

        # Define transform for the raster file
        transform = from_origin(west=lon.min(), north=lat.max(), xsize=lon_res, ysize=lat_res)

        # Prepare the output raster file path
        output_file = os.path.splitext(nc_file_path)[0] + '_processed.tif'

        # Save the data as a GeoTIFF file
        with rasterio.open(
            output_file,
            'w',
            driver='GTiff',
            height=total_moisture.shape[0],
            width=total_moisture.shape[1],
            count=1,
            dtype=np.float32,
            crs='EPSG:4326',  # Assuming data is in WGS84
            transform=transform
        ) as dst:
            dst.write(total_moisture.astype(np.float32), 1)

    print(f"Saved {output_file}")


In [ ]:
# Multiple .nc files transformation
# Directory containing the .nc files

import xarray as xr
import rasterio
from rasterio.transform import from_origin
import numpy as np
import os

input_directory = r"F:\Food Agility Data\Soil Features"

# Process each .nc file in the directory
for filename in os.listdir(input_directory):
    if filename.endswith('.nc'):
        file_path = os.path.join(input_directory, filename)
        print(file_path)
        process_nc_file(file_path)


In [3]:
file_path = "F://New_boundaries__Soil_data-12-2024-1_processed_processed.gpkg"

gdf = gpd.read_file(file_path)
print("GeoDataFrame loaded successfully.")
    
# List the columns
list_columns(gdf)
print(list_columns)


GeoDataFrame loaded successfully.
Columns in the GeoDataFrame:
1. OBJECTID
2. ID
3. year
4. crop_name
5. pred_prob
6. area_ha
7. yield
8. production
9. lga_name
10. Shape__Area
11. Shape__Length
12. Soil_Illite_30m_mean
13. CLY-1_mean
14. CLY-2_mean
15. CLY-3_mean
16. CLY-4_mean
17. CLY-5_mean
18. CLY-6_mean
19. CLY-7_mean
20. CLY-8_mean
21. CLY-9_mean
22. Sand-1_mean
23. Sand-2_mean
24. Sand-3_mean
25. Sand-4_mean
26. Sand-5_mean
27. Sand-6_mean
28. Sand-7_mean
29. Sand-8_mean
30. Sand-9_mean
31. Silt-1_mean
32. Silt-2_mean
33. Silt-3_mean
34. Silt-4_mean
35. Silt-5_mean
36. Silt-6_mean
37. Silt-7_mean
38. Silt-8_mean
39. Silt-9_mean
40. DUL-1_mean
41. DUL-2_mean
42. DUL-3_mean
43. DUL-4_mean
44. DUL-5_mean
45. DUL-6_mean
46. DUL-7_mean
47. DUL-8_mean
48. DUL-9_mean
49. SOF-1_mean
50. SOF-2_mean
51. SOF-3_mean
52. SOF-4_mean
53. SOF-5_mean
54. SOF-6_mean
55. SOF-7_mean
56. SOF-8_mean
57. SOF-9_mean
58. SOF-10_mean
59. SOF-11_mean
60. SOF-12_mean
61. SOF-13_mean
62. SOF-14_mean
63. SOF

In [16]:
with xr.open_dataset('H:/Food Agility Data/Soil Features/Soil Moisture/sm_pct_relative_2020.nc', decode_times=False) as ds:
    print(ds.dims) # <-- should show time, lat, lon
    print(ds['sm_pct'].dims) # <-- should explicitly show ('time', 'lat', 'lon')


FrozenMappingWarningOnValuesAccess({'time': 113, 'latitude': 681, 'longitude': 841, 'nv': 2})
('time', 'latitude', 'longitude')


In [30]:
import netCDF4 as nc
import numpy as np
import rasterio
from rasterio.transform import from_bounds
from datetime import datetime, timedelta

nc_file = "C:/Users/M.Ibrah/Downloads/sm_pct_variable.nc"
dataset = nc.Dataset(nc_file)

# Extract variable and time
sm_data = dataset.variables['sm_pct']
time_var = dataset.variables['time']
base_date = datetime(1900, 1, 1)
dates = np.array([base_date + timedelta(days=int(day)) for day in time_var[:]])

fill_value = -999

# Get spatial info
xmin, ymin, xmax, ymax = 111.975, -44.025, 154.025, -9.975
transform = from_bounds(xmin, ymin, xmax, ymax, 841, 681)
crs = 'EPSG:4326'

# Define years and months for extraction
years = [2020, 2021, 2022]
months = [5, 6, 7, 8, 9, 10]

for year in years:
    # Find band indices matching May-Oct of the target year
    band_indices = [i for i, date in enumerate(dates) if date.year == year and date.month in months]

    if not band_indices:
        print(f"No data found for year {year}.")
        continue

    monthly_bands = []

    with rasterio.open(f'NETCDF:"{nc_file}":sm_pct') as src:
        for band_idx in band_indices:
            band_data = src.read(band_idx + 1)  # Correct indexing

            # Replace fill_value with np.nan
            band_data = np.where(band_data == fill_value, np.nan, band_data)
            monthly_bands.append(band_data)

    monthly_bands = np.stack(monthly_bands)

    # Save individual monthly bands to raster
    with rasterio.open(
        f'C:/Users/M.Ibrah/Downloads/soil_moisture_{year}_variable.tif', 'w',
        driver='GTiff',
        height=monthly_bands.shape[1],
        width=monthly_bands.shape[2],
        count=len(band_indices),
        dtype=np.float32,
        crs=crs,
        transform=transform,
        nodata=np.nan
    ) as dst:
        for i, band in enumerate(monthly_bands, 1):
            dst.write(band, i)

    # Calculate mean ignoring NaNs
    mean_band = np.nanmean(monthly_bands, axis=0).astype(np.float32)

    # Save mean raster
    with rasterio.open(
        f'C:/Users/M.Ibrah/Downloads/sm_pct_mean_{year}_May_Oct_variable.tif',
        'w',
        driver='GTiff',
        height=mean_band.shape[0],
        width=mean_band.shape[1],
        count=1,
        dtype='float32',
        crs=crs,
        transform=transform,
        nodata=np.nan
    ) as dst_mean:
        dst_mean.write(mean_band, 1)

print("Extraction and saving completed correctly.")

C:\Users\M.Ibrah\AppData\Local\Temp\ipykernel_19216\4148359151.py:63: RuntimeWarning: Mean of empty slice
  mean_band = np.nanmean(monthly_bands, axis=0).astype(np.float32)


Extraction and saving completed correctly.
